# Agenda

1. Inheritance
2. Magic methods (`__del__`)
3. Object system
4. Metaclasses
5. Iterators etc. 

In [2]:

class Person:

    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
p1 = Person('name1')    
p2 = Person('name2')

print(p1.greet()) 
print(p2.greet()) 

class Employee(Person):    # Employee is-a Person, i.e., inherits from Person

    def __init__(self, name, id_number):
        # Person.__init__(self, name) 
        super().__init__(name)        # do what my parent does in __init__...
        self.id_number = id_number    # add my own things
            
e1 = Employee('emp1', 1)# e1 has __init__? no. Employee has __init__? Yes 
e2 = Employee('emp2', 2)

print(e1.greet()) # e1 has greet? No. Employee has greet? No. Person has greet? yes
print(e2.greet()) 
     

Hello, name1!
Hello, name2!
Hello, emp1!
Hello, emp2!


# Inheritance

All inheritance is based on the search for attributes. When we look for an attribute in a Python object (an instance, that is), Python searches in the following order:

- i -- the instance itself
- c -- the class of the instance (`type(i)`)
- p -- the parent of the class
- o -- `object`, the top object in the class

This means, in practice:

- If we have the same method in both a child class and a parent class, then we can remove the child class implementation, and rely on the parent class
- If we write a method in the child class, then that takes priority, and the parent class's method is never run.
- If we want to combine the method in the child class with the parent class, then we have a few options:
    1. Copy the code from the parent class into the child class. There are a number of problems with doing it this way -- not recommended.
    2. Call the parent method explicitly (`Class.method(self, arg1)`). This way, the parent class gets to run first, and then we add functionality in the child class.
    3. The most modern way is to use `super`, as in `super().method(arg1)`. We don't need to pass `self` here! Once again, we normally do this at first in the method, and then have more specific instructions in our method.
 


In [3]:
class First:
    def __init__(self, x):
        self.x = x

    def x2(self):
        return self.x * 2

class Second:
    def __init__(self, y):
        self.y = y

    def y3(self):
        return self.y * 3

class Third(First, Second):
    pass

In [4]:
# who does Person inherit from?  We can always check __bases__
Person.__bases__

(object,)

In [5]:
# What about Person's MRO (method resolution order)
Person.__mro__

(__main__.Person, object)

In [6]:
Employee.__bases__

(__main__.Person,)

In [7]:
Employee.__mro__

(__main__.Employee, __main__.Person, object)

In [8]:
First.__bases__

(object,)

In [9]:
First.__mro__

(__main__.First, object)

In [10]:
Second.__bases__

(object,)

In [11]:
Second.__mro__

(__main__.Second, object)

In [12]:
Third.__bases__

(__main__.First, __main__.Second)

In [13]:
Third.__mro__

(__main__.Third, __main__.First, __main__.Second, object)

In [14]:
# what happens when we create an instance of Third?

t = Third()

TypeError: First.__init__() missing 1 required positional argument: 'x'

In [15]:
t = Third(10)

In [16]:
vars(t)

{'x': 10}

In [17]:
t.x2()  # t has x2? No. Third has x2? No. First has x2? Yes...

20

In [18]:
t.y3()  # t has y3? No. Third has y3? No. First has y3? No. Second has y3? Yes

AttributeError: 'Third' object has no attribute 'y'

In [20]:
class BadClass(First, Third, Second):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases First, Third, Second

In [ ]:
class First:
    def __init__(self, x):
        self.x = x

    def x2(self):
        return self.x * 2

class Second:
    def __init__(self, y):
        self.y = y

    def y3(self):
        return self.y * 3

class Third(First, Second):
    def __init__(self, x, y):
        super().__init__(self, x)
        # First.__init__(self, x)
        # Second.__init__(self, y)

In [21]:
help(super)

Help on class super in module builtins:

class super(object)
 |  super() -> same as super(__class__, <first argument>)
 |  super(type) -> unbound super object
 |  super(type, obj) -> bound super object; requires isinstance(obj, type)
 |  super(type, type2) -> bound super object; requires issubclass(type2, type)
 |  Typical use to call a cooperative superclass method:
 |  class C(B):
 |      def meth(self, arg):
 |          super().meth(arg)
 |  This works for class methods too:
 |  class C(B):
 |      @classmethod
 |      def cmeth(cls, arg):
 |          super().cmeth(arg)
 |  
 |  Methods defined here:
 |  
 |  __get__(self, instance, owner=None, /)
 |      Return an attribute of instance, which is of type owner.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ---------------------

In [32]:
class First:
    def __init__(self, x):
        self.x = x

    def x2(self):
        return self.x * 2

class Second(First):
    def __init__(self, y):
        super().__init__(y)
        self.y = y

    def y3(self):
        return self.y * 3

class Third(Second):
    def __init__(self, x):
        super().__init__(x)

In [31]:
t = Third(17)

In [33]:
vars(t)

{'x': 17, 'y': 17}

In [34]:
object

object

In [35]:
str(t)

'<__main__.Third object at 0x11237db10>'

In [36]:
0x11237db10

4600617744